In [ ]:
### Enter topic of interest

topic = 'AI Research by Google'

topic = 'AI-powered Self-Service Analytics'


## Use CrewAI

- [GitHub](https://github.com/joaomdmoura/crewai/)
- [Docs](https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/)

### Quick Overview

Top-down approach
- Goal: What is the crew for? (define the use-case)
- Define all the Tasks required to achieve the goal? (including human review/action)
- Define and Assign the agent(s) required to complete each task
- Define Tools (maybe used by Agent/Task)
- Kickoff Crew process

In [1]:
from api_key_store import ApiKeyStore
s = ApiKeyStore()

openai_api_key = s.get_api_key(provider="OPENAI")
serp_api_key = s.get_api_key(provider="SerpApi")

In [2]:
s.api_providers

dict_keys(['OPENAI', 'MICROSOFT', 'GOOGLE', 'HUGGING_FACE', 'WOLFRAM_ALPHA', 'ANTHROPIC', 'COHERE', 'SerpApi', 'Browserless', 'DeepL'])

In [3]:
import os
os.environ["SERPER_API_KEY"] = serp_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [4]:
from crewai import Agent
from crewai_tools import SerperDevTool

### Step 1: Assemble Your Agents

#### Define Tools

In [6]:
search_tool = SerperDevTool()

#### Define Agents

In [7]:
# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True
)

In [8]:
# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)

### Step 2: Define the Tasks

In [9]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

### Step 3: Form the Crew

In [10]:
from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True, # False,  # 
  cache=True, # False,   # 
  max_rpm=50,
  # share_crew=True
)


Issue:
```
OperationalError                          Traceback (most recent call last)
File ~/.local/lib/python3.10/site-packages/sqlalchemy/engine/base.py:1967, in Connection._exec_single_context(self, dialect, context, statement, parameters)
   1966     if not evt_handled:
-> 1967         self.dialect.do_execute(
   1968             cursor, str_statement, effective_parameters, context
   1969         )
   1971 if self._has_events or self.engine._has_events:
```
root-cause:

```
which jupyter
/home/gongai/.local/bin/jupyter

```

### Step 4: Kick It Off

In [11]:
from time import time
ts_start = time()

In [12]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': topic})

ts_stop = time()
print(result)



> Entering new CrewAgentExecutor chain...
To begin, I need to gather as much information as possible about the latest trends in AI-powered Self-Service Analytics, their pros and cons, market opportunities, and potential risks. I'll use the internet search tool to find the most relevant and recent articles, blog posts, and research papers on this topic.

Action: Search the internet
Action Input: {"search_query": "Latest trends in AI-powered Self-Service Analytics"} 


Search results: Title: AI-Driven Self-Service analytics: How AI Impacts Industries and ...
Link: https://insightbase.ai/blog/ai-driven-self-service-analytics
Snippet: Further in our article, we will discuss more in depth about the top industries dominated by AI-driven self-service analytics tools, the impact of using ...
---
Title: Self-Service Analytics: The Future of Data & BI Powered by AI - Tellius
Link: https://www.tellius.com/future-of-self-service-analytics-reporting-ai-ml/
Snippet: Today, self-service analytics i

In [13]:
elapsed_time = ts_stop - ts_start

In [14]:
print(f"""Done researching topic: "{topic}" in {elapsed_time:.2f} sec""")

Done researching topic: AI-powered Self-Service Analytics in 553.21 sec
